In [1]:
# 🧹 메모리 정리 및 환경 설정
import gc
import os
import sys
from dotenv import load_dotenv

def cleanup_memory():
    """메모리 정리 및 가비지 컬렉션"""
    # 기존 C1 retrieval system이 있다면 정리
    if 'c1_retrieval' in globals():
        try:
            c1_retrieval.close()
            del globals()['c1_retrieval']
            print("✅ 기존 C1 검색 시스템 연결 해제")
        except:
            pass
    
    # 기존 C2 retrieval system이 있다면 정리
    if 'c2_retrieval' in globals():
        try:
            c2_retrieval.close()
            del globals()['c2_retrieval']
            print("✅ 기존 C2 검색 시스템 연결 해제")
        except:
            pass

    # 기존 C3 retrieval system이 있다면 정리
    if 'c3_retrieval' in globals():
        try:
            c3_retrieval.close()
            del globals()['c3_retrieval']
            print("✅ 기존 C3 검색 시스템 연결 해제")
        except:
            pass
    
    # 기존 C4 retrieval system이 있다면 정리
    if 'c4_retrieval' in globals():
        try:
            c4_retrieval.close()
            del globals()['c4_retrieval']
            print("✅ 기존 C4 검색 시스템 연결 해제")
        except:
            pass
    
    # 모듈 캐시 정리 (config 모듈 등 새로고침)
    modules_to_reload = ['config', 'c1_retrieval', 'c2_retrieval', 
                        'prompts.c1_prompt', 'prompts.c2_prompt']
    
    for module_name in modules_to_reload:
        if module_name in sys.modules:
            try:
                del sys.modules[module_name]
                print(f"🔄 {module_name} 모듈 캐시 정리")
            except:
                pass
    
    # 가비지 컬렉션 수행
    collected = gc.collect()
    print(f"🧹 가비지 컬렉션 완료: {collected}개 객체 정리")
    
    # 캐시 정보 확인
    try:
        cache_file = "embedding_cache.json"
        if os.path.exists(cache_file):
            size_mb = os.path.getsize(cache_file) / 1024 / 1024
            print(f"📦 임베딩 캐시 크기: {size_mb:.2f} MB")
    except:
        pass
    
    print("✨ 메모리 정리 완료!")

# 환경 변수 로드
load_dotenv()

# 필수 환경 변수 확인
required_vars = ['OPENAI_API_KEY', 'NEO4J_URI', 'NEO4J_USER', 'NEO4J_PASSWORD']
missing_vars = [var for var in required_vars if not os.getenv(var)]

cleanup_memory()

🧹 가비지 컬렉션 완료: 0개 객체 정리
📦 임베딩 캐시 크기: 46.81 MB
✨ 메모리 정리 완료!


In [2]:
# 🤖 LangGraph Agent RAG 시스템 통합 테스트 - 비교 버전
from pipeline import run_pipeline
from IPython.display import display, HTML, Markdown
import json
from nodes.user_question import describe_image

def display_comparison_results(result: dict, query: str, image_path: str = None):
    """파이프라인 실행 결과를 비교하여 시각적으로 표시"""
    
    # 이미지가 있었다면 표시
    if image_path:
        print(f"🖼️ 입력 이미지: {image_path}")
        # 이미지 설명이 포함된 경우 표시
        input_text_with_image = result.get('input_text_with_image', '')
        if input_text_with_image and input_text_with_image != query:
            print(f"📸 파이프라인에서 생성된 이미지 설명 + 쿼리:")
            print(f"   → {input_text_with_image}")
            print()
    
    # 기본 정보 표시
    task_type = result.get('task_type', 'Unknown')
    initial_response = result.get('initial_response', '초기 답변이 없습니다.')
    final_response = result.get('final_response', '최종 답변이 없습니다.')
    initial_results = result.get('initial_search_results', [])
    final_results = result.get('final_search_results', [])
    debug_info = result.get('debug_info', {})
    
    # 최고 점수 라운드 정보 가져오기
    final_best_round = result.get('final_best_round', 3)
    final_best_score = result.get('final_best_score', 0)
    final_best_top_k = result.get('final_best_top_k', len(final_results))
    
    print(f"🚀 LangGraph Agent RAG 시스템 비교 결과")
    print("=" * 80)
    print(f"📝 사용자 질문: {query}")
    print(f"🎯 분류된 태스크: {task_type}")
    
    # 실행 통계 표시
    if debug_info:
        classification_info = debug_info.get('classification', {})
        reflection_history = debug_info.get('reflection_history', [])
        generation_info = debug_info.get('generation', {})
        
        print(f"\n📊 실행 통계:")
        if classification_info:
            print(f"   • 태스크 분류 신뢰도: {classification_info.get('confidence', 0):.1f}%")
            print(f"   • 분류 이유: {classification_info.get('reason', 'N/A')}")
        
        if reflection_history:
            print(f"   • Reflection 반복 횟수: {len(reflection_history)}회")
            for i, reflection in enumerate(reflection_history, 1):
                print(f"     - 라운드 {i}: {reflection.get('score', 0):.1f}점 (Top-K: {reflection.get('top_k', 0)})")
        
        if generation_info:
            print(f"   • 초기 칵테일: {generation_info.get('initial_cocktails_count', 0)}개")
            print(f"   • 최종 칵테일: {generation_info.get('final_cocktails_count', 0)}개 (Round {final_best_round})")
            print(f"   • 최고 점수: {final_best_score:.1f}점 (Round {final_best_round}, Top-K: {final_best_top_k})")
    
    print("=" * 80)
    
    # 1. Round 1 답변 HTML 블록 (노란색)
    display(HTML(f"""
    <div style="background-color: #fff3cd; padding: 25px; border-radius: 12px; margin: 20px 0; border-left: 6px solid #ffc107;">
        <h2 style="color: #856404; margin-top: 0; margin-bottom: 15px;">🥉 Round 1 답변 ({len(initial_results)}개 칵테일)</h2>
        <div style="background-color: #fefefe; padding: 15px; border-radius: 8px; margin-top: 15px;">
            <div style="white-space: pre-wrap; font-family: 'Segoe UI', Arial, sans-serif; line-height: 1.7; color: #212529; font-size: 15px;">{initial_response}</div>
        </div>
    </div>
    """))
    
    # 2. 구분선 및 개선 표시
    improvement_text = ""
    if len(final_results) > len(initial_results):
        improvement_text = f"<span style='color: #28a745; font-weight: bold;'>+{len(final_results) - len(initial_results)}개 더 많은 칵테일</span>"
    elif len(final_results) == len(initial_results):
        improvement_text = f"<span style='color: #6c757d;'>동일한 개수, 최고 품질 선택 (Round {final_best_round})</span>"
    else:
        improvement_text = f"<span style='color: #dc3545; font-weight: bold;'>{len(final_results) - len(initial_results):+d}개 칵테일</span>"
    
    display(HTML(f"""
    <div style="text-align: center; margin: 30px 0;">
        <hr style="border: 2px solid #007bff; margin-bottom: 15px;">
        <h2 style="color: #007bff; margin: 10px 0;">⚡ REFLECTION 개선 효과 ⚡</h2>
        <p style="color: #6c757d; font-size: 16px;">{improvement_text}</p>
        <hr style="border: 2px solid #007bff; margin-top: 15px;">
    </div>
    """))
    
    # 3. 최종 답변 HTML 블록 (파란색) - 최고 점수 라운드 정보 표시
    display(HTML(f"""
    <div style="background-color: #d1ecf1; padding: 25px; border-radius: 12px; margin: 20px 0; border-left: 6px solid #17a2b8;">
        <h2 style="color: #0c5460; margin-top: 0; margin-bottom: 15px;">🏆 최종 답변 (Round {final_best_round} - {len(final_results)}개 칵테일, {final_best_score:.1f}점)</h2>
        <div style="background-color: #fefefe; padding: 15px; border-radius: 8px; margin-top: 15px;">
            <div style="white-space: pre-wrap; font-family: 'Segoe UI', Arial, sans-serif; line-height: 1.7; color: #212529; font-size: 15px;">{final_response}</div>
        </div>
    </div>
    """))
    
    # 4. 개선 요약 표시 - 최고 점수 기준으로 계산
    if debug_info and reflection_history:
        first_score = reflection_history[0].get('score', 0) if reflection_history else 0
        best_score = final_best_score  # 최고 점수 사용
        score_change = best_score - first_score
        
        change_color = "#28a745" if score_change >= 0 else "#dc3545"
        change_icon = "📈" if score_change >= 0 else "📉"
        
        # 점수 변화가 없는 경우 (최고 점수가 초기 점수와 같음)
        if score_change == 0:
            change_icon = "🎯"
            change_color = "#17a2b8"
        
        display(HTML(f"""
        <div style="background-color: #f8f9fa; padding: 20px; border-radius: 8px; margin: 20px 0; border: 1px solid #dee2e6;">
            <h3 style="color: #495057; margin-top: 0;">📊 개선 요약</h3>
            <div style="display: flex; justify-content: space-between; align-items: center;">
                <div>
                    <strong>칵테일 개수:</strong> {len(initial_results)}개 → {len(final_results)}개 
                    <span style="color: {change_color};">({len(final_results) - len(initial_results):+d}개)</span>
                </div>
                <div>
                    <strong>품질 점수:</strong> {first_score:.1f}점 → {best_score:.1f}점 
                    <span style="color: {change_color};">{change_icon} ({score_change:+.1f}점)</span>
                </div>
            </div>
            <div style="margin-top: 10px; font-size: 14px; color: #6c757d;">
                💡 <strong>선택된 라운드:</strong> Round {final_best_round} (Top-K: {final_best_top_k}) - 최고 품질 결과
            </div>
        </div>
        """))
    
    # 5. 디버그 정보 표시 (접을 수 있는 형태)
    if debug_info:
        debug_json = json.dumps(debug_info, indent=2, ensure_ascii=False)
        display(HTML(f"""
        <details style="margin: 20px 0;">
            <summary style="cursor: pointer; font-weight: bold; color: #666; padding: 10px; background-color: #f8f9fa; border-radius: 5px;">
                🔍 상세 디버그 정보 (클릭해서 보기)
            </summary>
            <pre style="background-color: #f8f9fa; padding: 15px; border-radius: 5px; overflow-x: auto; margin-top: 10px; font-size: 12px;">{debug_json}</pre>
        </details>
        """))

class CocktailRAG:
    """간편 칵테일 RAG 시스템 인터페이스"""
    
    def __init__(self):
        self.image = None
        self.query = ""
    
    def run(self):
        """설정된 image와 query로 RAG 시스템 실행"""
        if not self.query:
            print("⚠️ query를 설정해주세요!")
            print("사용법:")
            print("  rag.image = '/path/to/image.jpg'  # 이미지가 있는 경우")
            print("  rag.query = '질문 내용'")
            print("  rag.run()")
            return None
        
        print("🧪 LangGraph Agent RAG 시스템 실행")
        print("=" * 60)
        
        # 이미지 경로만 표시 (파이프라인에서 실제 설명 생성)
        if self.image:
            print(f"🖼️ 이미지 경로: {self.image}")
        
        # 파이프라인 실행
        result = run_pipeline(self.query, self.image)
        
        # 결과 표시
        display_comparison_results(result, self.query, self.image)
        
        return result

# 전역 인스턴스 생성
rag = CocktailRAG()

print("✅ LangGraph Agent RAG 시스템 함수가 로드되었습니다!")
print("\n💡 사용법:")
print("  rag.image = '/path/to/image.jpg'  # 이미지가 있는 경우 (없으면 None)")
print("  rag.query = '질문 내용'")
print("  rag.run()")

✅ LangGraph Agent RAG 시스템 함수가 로드되었습니다!

💡 사용법:
  rag.image = '/path/to/image.jpg'  # 이미지가 있는 경우 (없으면 None)
  rag.query = '질문 내용'
  rag.run()


In [3]:
rag.image = '/Users/yujin/Desktop/cocktail_rec_agentrag/langgraph/image/mojito.jpeg'  # 이미지 없이 텍스트만으로 테스트
rag.query = "이 칵테일과 비슷한 칵테일 3개만 추천하고, 색/가니시/글라스 근거를 한 줄씩 적어줘."  # 간단한 질문
rag.run()  # 실행

🧪 LangGraph Agent RAG 시스템 실행
🖼️ 이미지 경로: /Users/yujin/Desktop/cocktail_rec_agentrag/langgraph/image/mojito.jpeg
🚀 파이프라인 시작
📝 질문: 이 칵테일과 비슷한 칵테일 3개만 추천하고, 색/가니시/글라스 근거를 한 줄씩 적어줘.
🖼️ 이미지: /Users/yujin/Desktop/cocktail_rec_agentrag/langgraph/image/mojito.jpeg
🖼️ 이미지 설명 생성 중: /Users/yujin/Desktop/cocktail_rec_agentrag/langgraph/image/mojito.jpeg
✅ 이미지 설명 생성 완료
🖼️ 이미지 설명 생성 중: /Users/yujin/Desktop/cocktail_rec_agentrag/langgraph/image/mojito.jpeg
✅ 이미지 설명 생성 완료
🤖 태스크 분류 중...
📝 쿼리: The cocktail depicted is a refreshing mojito, showcasing a tall, clear glass filled with ice cubes that reflect light. The drink features vibrant green mint leaves and circular slices of cucumber, which provide a contrasting texture against the sparkling liquid. The composition is enhanced by a blurred background with soft candlelight, creating a warm ambiance. Notably, the cocktail's vibrant colors and fresh ingredients contribute to its visually appealing presentation, invoking a sense of refreshment and vitality

{'user_query': {'text': '이 칵테일과 비슷한 칵테일 3개만 추천하고, 색/가니시/글라스 근거를 한 줄씩 적어줘.',
  'image': '/Users/yujin/Desktop/cocktail_rec_agentrag/langgraph/image/mojito.jpeg'},
 'input_text': '이 칵테일과 비슷한 칵테일 3개만 추천하고, 색/가니시/글라스 근거를 한 줄씩 적어줘.',
 'input_image': '/Users/yujin/Desktop/cocktail_rec_agentrag/langgraph/image/mojito.jpeg',
 'task_type': 'C1',
 'task_confidence': 90.0,
 'task_reason': '색상과 시각적 요소에 대한 설명이 포함되어 있어 C1이 적합',
 'input_text_with_image': "The cocktail depicted is a refreshing mojito, showcasing a tall, clear glass filled with ice cubes that reflect light. The drink features vibrant green mint leaves and circular slices of cucumber, which provide a contrasting texture against the sparkling liquid. The composition is enhanced by a blurred background with soft candlelight, creating a warm ambiance. Notably, the cocktail's vibrant colors and fresh ingredients contribute to its visually appealing presentation, invoking a sense of refreshment and vitality. 이 칵테일과 비슷한 칵테일 3개만 추천하고, 색/가니시/글라